**water shortage likelihood prediction**

Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings


Load and view aquastat data set

In [ ]:
data = pd.read_csv('https://mda-project-poland.s3.eu-west-3.amazonaws.com/aquastat_data.csv') 
data = data.pivot_table(index = "Area", columns = "Variable Name", values= "Value")
data.head()

dataset of country location（latitude and longitude)

In [ ]:
location=pd.read_csv('https://mda-project-poland.s3.eu-west-3.amazonaws.com/location.csv')
location.head()

combine two datasets

In [ ]:
loc1 = location.loc[:, [ "Area","Latitude","Longitude"]]
loc1.drop_duplicates(inplace=True)
data =loc1.merge(data, on="Area", how="left")
data

Calculate the new variables

In [ ]:
data['% of total country area irrigated (%)']=data['% of total country area cultivated']*data['% of cultivated land irrigated [harvested crop]']/100
data['Industry, value added (% GDP)']=data['Industry, value added to GDP']/data['Gross Domestic Product (GDP)']*100
data['Services, value added (% GDP)']=data['Services, value added to GDP']/data['Gross Domestic Product (GDP)']*100
data['water resource per capita']=((data['Total renewable water resources']*1000000)-(data['Environmental Flow Requirements']*1000000))/data['Total population']

In [ ]:
print(data['water resource per capita'].describe())

In [ ]:
data.isnull().any()

## Data Preprocessing and exploration


In [ ]:
df = data.copy()
#Data types
datadict = pd.DataFrame(df.dtypes)
#Missing values
datadict['MissingVal'] = df.isnull().sum()
#Unique values
datadict['NUnique']=df.nunique()
#Count of variable
datadict['Count']=df.count()
#Rename 0 to datatype
datadict = datadict.rename(columns={0:'DataType'})
datadict

the distribution of water shortage index

In [ ]:
df['water resource per capita'].hist(grid=False, bins=20)

right skewed

dichotomization:

water shortage: water resource per capita<1700 m^3 per person per year

In [ ]:
df['c']=0
df['c'][df['water resource per capita']< 1700]=1

Delete rows with missing values of water shortage index

replace NaN in the predictors with mean

In [ ]:
df = df.dropna(axis=0,subset=['water resource per capita'])
df = df.fillna(df.mean())
df.shape

In [ ]:
g = sns.catplot(
    data=df, kind="bar", x = "Area", y='water resource per capita', 
    ci="sd", palette="icefire", alpha=.9, height=12)
g.set_xticklabels(rotation=53)
g.set_axis_labels( "Area", 'water resource per capita')

In [ ]:
data['water resource per capita'].describe()

## Fit a logistic regression model

Variable selection procedure: backward elimination based on p-value 

In [ ]:
x_columns=['Latitude',
    'Longitude',
    '% of total country area cultivated',
    '% of total country area irrigated (%)',
    'Long-term average annual precipitation in depth',
    'Population density',
    'GDP per capita',
    'Agriculture, value added (% GDP)',
    'Industry, value added (% GDP)',
    'Services, value added (% GDP)',]
x = df[x_columns]
y = df['c']
x = sm.add_constant(x)

In [ ]:
model = sm.Logit(y,x)
result = model.fit()
result.summary()

remove longitude

In [ ]:
x_columns.remove('Longitude')
x = df[x_columns]
y = df['c']
x = sm.add_constant(x)
model = sm.Logit(y,x)
result = model.fit()
result.summary()

remove GDP per captia

In [ ]:
x_columns.remove('GDP per capita')
x=df[x_columns]
y=df['c']
x = sm.add_constant(x)
model = sm.Logit(y,x)
result = model.fit()
result.summary()

remove Services, value added (% GDP)

In [ ]:
x_columns.remove('Services, value added (% GDP)')
x=df[x_columns]
y=df['c']
x = sm.add_constant(x)
model = sm.Logit(y,x)
result = model.fit()
result.summary()

remove Industry, value added (% GDP)

In [ ]:
x_columns.remove('Industry, value added (% GDP)')
x=df[x_columns]
y=df['c']
x = sm.add_constant(x)
model = sm.Logit(y,x)
result = model.fit()
result.summary()

remove % of total country area cultivated

In [ ]:
x_columns.remove('% of total country area cultivated')
x=df[x_columns]
x = sm.add_constant(x)
model = sm.Logit(y,x)
result = model.fit()
result.summary()

predicted accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y,round(result.predict(x)))

the likelihood of a water shortage 

In [ ]:
df['likelihood'] = result.predict(x)

## Divide the water shortage indicator into 4 risk levels

In [ ]:
category = pd.cut(aquastat['water resource per capita'],bins=[0,500,1000,1700,220071],labels=['3','2','1','0'])
aquastat.insert(21,'c of shortage',category)

In [ ]:
data = data.dropna(axis=0,subset=['water resource per capita'])
data = data.fillna(df.mean())

In [ ]:
data['c of shortage'].unique()

In [ ]:
x_columns=['Latitude',
    'Longitude',
    '% of total country area cultivated',
    '% of total country area irrigated (%)',
    'Long-term average annual precipitation in depth',
    'Population density',
    'GDP per capita',
    'Agriculture, value added (% GDP)',
    'Industry, value added (% GDP)',
    'Services, value added (% GDP)']
x=data[x_columns]
y=data['c of shortage']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes
from sklearn.neighbors import KNeighborsClassifier

pipe = Pipeline([('classifier', RandomForestClassifier())])
param_grid = [
    {'classifier': [RandomForestClassifier()],
    'classifier__n_estimators':[50,100],'classifier__max_features': [1, 2, 3, 4],
    'classifier__min_samples_leaf':[1,2,3]},
    {'classifier': [LogisticRegression(max_iter=1000)]},
    {'classifier': [naive_bayes.GaussianNB()]},
    {'classifier': [KNeighborsClassifier()],
     'classifier__leaf_size': [10,20,30],
     'classifier__n_neighbors': [3,5,7,10]}
    ]

X_train, X_test, y_train, y_test = train_test_split(
    x, y, random_state=42)

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))